In [1]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymysql

     -------------------------------------- 44.8/44.8 kB 442.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql

from sqlalchemy import create_engine

import pandas as pd

 

import getpass

In [5]:
password = getpass.getpass()


········


In [6]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [10]:
# Obtén una lista de nombres de tablas
table_names = engine.table_names()

# Imprime los nombres de las tablas
for table_name in table_names:
    print(table_name)
    

actor
address
category
city
country
customer
film
film_actor
film_category
film_text
films_2020
inventory
language
payment
rental
rentals_june
rentals_may
staff
store


C:\Users\raull\AppData\Local\Temp\ipykernel_27988\2880968007.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


   actor_id first_name     last_name         last_update
0         1   PENELOPE       GUINESS 2006-02-15 04:34:33
1         2       NICK      WAHLBERG 2006-02-15 04:34:33
2         3         ED         CHASE 2006-02-15 04:34:33
3         4   JENNIFER         DAVIS 2006-02-15 04:34:33
4         5     JOHNNY  LOLLOBRIGIDA 2006-02-15 04:34:33


In [13]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [19]:
# 1. Crea una consulta para extraer la información relevante
query = """
SELECT
    f.film_id,
    f.title,
    f.rental_duration,
    f.rental_rate,
    f.length,
    f.rating,
    c.name AS category_name,
    IFNULL((
        SELECT 1
        FROM rental r
        JOIN inventory i ON r.inventory_id = i.inventory_id
        WHERE r.rental_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
            AND r.rental_date < CURDATE()
            AND i.film_id = f.film_id
        LIMIT 1
    ), 0) AS rented_last_month
FROM
    film f
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
"""

In [20]:
# 2. Lee los datos en un dataframe de Pandas
df = pd.read_sql(query, engine)  # Asegúrate de tener la conexión establecida al motor de la base de datos

In [21]:
df = pd.get_dummies(df, columns=['rating', 'category_name'])

In [22]:
# 4. Crea una consulta para obtener la lista de películas y si fueron alquiladas el mes pasado
target_query = """
SELECT
    film_id,
    IFNULL((
        SELECT 1
        FROM rental r
        JOIN inventory i ON r.inventory_id = i.inventory_id
        WHERE r.rental_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
            AND r.rental_date < CURDATE()
            AND i.film_id = film.film_id
        LIMIT 1
    ), 0) AS rented_last_month
FROM
    film
"""

In [23]:
target_df = pd.read_sql(target_query, engine)

In [24]:

# 5. Crea el modelo de regresión logística
X = df.drop(['film_id', 'title', 'rented_last_month'], axis=1)
y = target_df['rented_last_month']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [25]:

# 6. Evalúa los resultados
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [26]:
# Verificar la distribución de las clases en el conjunto de datos objetivo
class_counts = target_df['rented_last_month'].value_counts()
print(class_counts)

0    1000
Name: rented_last_month, dtype: int64
